# Suo

In [1]:
import warnings
import os
import sys
import gc
import warnings

In [2]:
import anndata as ad
import scanpy as sc
import copy
import torch
from pathlib import Path
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import numpy as np
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.stats
from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection
import scvi

sys.path.append("/home/icb/kemal.inecik/work/codes/tardis")
import tardis
tardis.config = tardis.config_server

sc.settings.verbosity = 3

In [6]:
torch.cuda.is_available()

False

In [4]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
_rcparams_path = "/home/icb/kemal.inecik/work/codes/tardis/training/local/figures/rcparams.pickle"
with open(_rcparams_path, 'rb') as file:
    _rcparams = pickle.load(file)
plt.rcParams.update(_rcparams)

In [9]:
adata = ad.read_h5ad("/home/icb/kemal.inecik/lustre_workspace/tardis_data/processed/dataset_complete_Suo_invae.h5ad")
obsms = [i for i in adata.obsm if i not in ['Unintegrated', 'X_pca', 'harmony']]
adata

AnnData object with n_obs × n_vars = 841922 × 8192
    obs: 'sample_ID', 'organ', 'age', 'cell_type', 'sex', 'sex_inferred', 'concatenated_integration_covariates', 'integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse', 'n_genes'
    uns: 'rank_genes_groups'
    obsm: 'Unintegrated', 'X_pca', 'harmony', 'inveriant_1', 'inveriant_3', 'inveriant_5'

In [10]:
biocons = BioConservation()
batchcor = BatchCorrection(
    silhouette_batch=False,
    ilisi_knn=False,
    kbet_per_label=False,
    graph_connectivity=False,
    pcr_comparison=False
)

In [11]:
def get_results(self, min_max_scale: bool = True) -> pd.DataFrame:
    _LABELS = "labels"
    _BATCH = "batch"
    _X_PRE = "X_pre"
    _METRIC_TYPE = "Metric Type"
    _AGGREGATE_SCORE = "Aggregate score"
    
    df = self._results.transpose()
    df.index.name = "Embedding"
    df = df.loc[df.index != _METRIC_TYPE]
    if min_max_scale:
        # Use sklearn to min max scale
        df = pd.DataFrame(
            MinMaxScaler().fit_transform(df),
            columns=df.columns,
            index=df.index,
        )
    df = df.transpose()
    df[_METRIC_TYPE] = self._results[_METRIC_TYPE].values

    # Compute scores
    per_class_score = df.groupby(_METRIC_TYPE).mean().transpose()
    # This is the default scIB weighting from the manuscript
    # per_class_score["Total"] = 0.4 * per_class_score["Batch correction"] + 0.6 * per_class_score["Bio conservation"]
    df = pd.concat([df.transpose(), per_class_score], axis=1)
    df.loc[_METRIC_TYPE, per_class_score.columns] = _AGGREGATE_SCORE
    return df

In [ ]:
print(adata, flush=True)
print("starting scvi scib", flush=True)
bm = Benchmarker(
    adata=adata,
    batch_key="concatenated_integration_covariates",
    label_key="cell_type",
    embedding_obsm_keys=obsms,
    pre_integrated_embedding_obsm_key="Unintegrated",  # equals to X_pca
    bio_conservation_metrics=biocons,
    batch_correction_metrics=batchcor,
    n_jobs=-1,
)

In [ ]:
bm.benchmark()

Computing neighbors: 100%|██████████| 1/1 [03:19<00:00, 199.18s/it]

Computing neighbors: 100%|██████████| 1/1 [03:19<00:00, 199.18s/it]

Embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

Metrics:   0%|          | 0/10 [00:00<?, ?it/s, Batch correction: silhouette_batch]

...

Metrics:  40%|████      | 4/10 [11:47<17:37, 176.19s/it, Batch correction: graph_connectivity]

Metrics:  40%|████      | 4/10 [11:47<17:37, 176.19s/it, Batch correction: pcr_comparison]    

Metrics:  50%|█████     | 5/10 [11:55<09:37, 115.45s/it, Batch correction: pcr_comparison]

Embeddings: 100%|██████████| 1/1 [11:55<00:00, 715.57s/it]

Embeddings: 100%|██████████| 1/1 [11:55<00:00, 715.57s/it]

In [ ]:
df = get_results(bm, min_max_scale=False)

In [ ]:
df

In [ ]:
pickle_path = os.path.join(tardis.config.io_directories["temp"], "latent", "suo_scib_invae_bio.pickle")
pickle_path

In [ ]:
df.to_pickle(pickle_path)